<a href="https://colab.research.google.com/github/jasonadu/gpt_in_practice/blob/main/2.%20%E6%8F%90%E7%A4%BA%E8%AF%8D%E6%8A%80%E5%B7%A7%EF%BD%9C%E8%8E%B7%E5%BE%97%E4%BB%A3%E7%A0%81%E5%8F%8B%E5%A5%BD%E7%9A%84%E5%A4%A7%E6%A8%A1%E5%9E%8B%E5%9B%9E%E5%A4%8D%20-%20Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 回顾

In [ ]:
deployment="gpt4" #在azure中的deployment name
model="gpt-4"

In [ ]:
def translate(text):
    messages = []
    messages.append( {"role": "system",
                      "content": "You are a translator. Please, translate the user's request to English."})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

print(encoding)

chinese = """在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。"""
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))

print(f"chinese:{chinese} ; {num_of_tokens_in_chinese} tokens\n")
print(f"english:{english} ; {num_of_tokens_in_english} tokens\n")

<Encoding 'cl100k_base'>
chinese:在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。 ; 35 tokens

english:Before the future arrives, there always needs to be someone to create it, and that person should be us. ; 22 tokens



# 常见应用场景

## 意图识别

In [1]:
import os
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('DASHSCOPE_API_KEY'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": """
        Recognize the intent from the user's input.
        """},
        {"role": "user", "content": "提醒我明早8点有会议"},
    ]
)

print(completion.choices[0].message.content)

好的，我将为您设置一个明天早上的8点会议提醒。请问还有其他需要补充的信息吗？例如会议的主题或地点，我可以帮您记录下来。


## 生成SQL

In [3]:
import openai, os

system_prompt =  """  You are a software engineer, you can anwser the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score]
                  table "courses" with the columns [id, name]
                  """

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"},
    ]
)

print(completion.choices[0].message.content)

要计算所有学生在英语课程中的平均成绩，我们需要进行以下步骤：

1. 在 `courses` 表中找到英语课程的 `id`。
2. 在 `students` 表中筛选出该 `course_id` 的记录。
3. 计算这些记录的 `score` 的平均值。

以下是实现的 SQL 查询：

```sql
SELECT AVG(students.score) AS average_score
FROM students
JOIN courses ON students.course_id = courses.id
WHERE courses.name = '英语';
```

### 解释：
- `JOIN` 用于将 `students` 和 `courses` 表连接起来。
- `WHERE courses.name = '英语'` 筛选出英语课程。
- `AVG(students.score)` 计算英语课程的平均成绩。

运行此查询后，你将得到所有学生在英语课程上的平均分数。


## 生成代码友好的提示词

### 规范输出的格式

In [4]:
completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content":  """
          Recognize the intent from the user's input and format output as JSON string.
        The output JSON string includes: "intention", "paramters" """},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
)

print(completion.choices[0].message.content)

```json
{
  "intention": "设置提醒",
  "paramters": {
    "时间": "明早8点",
    "事件": "会议"
  }
}
```


In [5]:
system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content":  system_prompt},
        {"role": "user", "content": "列出英语课程成绩大于80分的学生, 返回结果只包括SQL"},
    ]
)

print(completion.choices[0].message.content)

SELECT s.id, s.name, s.score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = '英语' AND s.score > 80;


### 文本规范异常输出的格式

In [6]:
system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

system_prompt_with_negative =  """
You are a software engineer, you can write a SQL string as the anwser according to the user request.
Also, when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request"
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

In [7]:
prompt = system_prompt
# prompt = system_prompt_with_negative
completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content":  prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ]
)

print(completion.choices[0].message.content)

根据你的描述，表结构中并没有“年龄”字段，因此无法直接查询年龄大于13的学生。通常“年龄”信息可能在 `students` 表中，也可能在另外的表中（如 `users` 或 `profiles` 表），但根据你提供的信息，我们目前没有该字段。

如果你是想根据出生日期（birthday）字段来计算年龄，并筛选年龄大于13的学生，请提供该字段。否则，请确认是否有其他字段或表包含年龄信息。

如果你确实有误，且实际需求是列出学生信息（如成绩、所选课程等），请重新描述你的需求。

假设你有一个 `birthdays` 表或字段，可以参考如下 SQL（假设字段为 `birthday`）：

```sql
SELECT *
FROM students
WHERE DATE_PART('year', AGE(birthday)) > 13;
```

如果你能提供更多字段信息，我可以给出更准确的 SQL。


In [8]:
# prompt = system_prompt
prompt = system_prompt_with_negative
completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content":  prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ]
)

print(completion.choices[0].message.content)

invalid request
